In [1]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from matplotlib import pyplot as plt

In [6]:
source_dir = '/drive/colab/sr/images-cropped-196/'  # images to process
save_dir = '/drive/colab/sr/images-blurred-downsampled-96/'  # processed images

In [27]:
list_ds = tf.data.Dataset.list_files(source_dir + '*.jpg', shuffle=False)

In [28]:
import os

def save_img(img, title):
    try:
        keras.utils.save_img(save_dir + title, img)
    except Exception:
        print(title)  # print offenders
        raise Exception


def blur_resize_image(filename):
    # get the image title (name.ext)
    parts = tf.strings.split(filename, os.sep)
    title = bytes.decode(parts[-1].numpy())

    # skip non .jpg files
    if title.split('.')[-1] != 'jpg':
        return

    # skip files that have already been processed
    if os.path.isfile(save_dir + title):
        return

    try:
        img = tf.io.read_file(filename)
        img = tf.io.decode_jpeg(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tfa.image.gaussian_filter2d(img, filter_shape=(3,3))
        img = tf.image.resize(
            img,
            (96, 96),
            method=tf.image.ResizeMethod.AREA,
            preserve_aspect_ratio=False,
            antialias=False,
        )
    except Exception:
        print(title)   # print offenders
        raise Exception

    save_img(img, title)


In [10]:
# # for testing blur_image

# file_path = next(iter(list_ds))
# blur_resize_image(file_path)

In [29]:
for file_path in iter(list_ds):
    blur_resize_image(file_path)